# Tugas Lab 4

In [1]:
print('-'*42+'\nTugas Lab 4 PengCit - Kelompok Apa\n'+'-'*42)

------------------------------------------
Tugas Lab 4 PengCit - Kelompok Apa
------------------------------------------


### Feature Extraction

Color : R, G, B, S

Texture : Contrast, ASM, Homogeneity

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import sys, math

In [3]:
# For google colab purpose
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import glob
from skimage import io, color, util, feature
from skimage.transform import resize

# Ubah sesuai kebutuhan
workdir = '/content/drive/My Drive/TL4 PengCit/'

## Feature Model 1 : RGB + S

In [5]:
train_shape_feature = [["avg_r", "avg_g", "avg_b", "avg_s", "type_label"]]

# CUACA TRAIN FEATURE MODEL 1
def generate_train(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Train/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])
    R = RGB[:,:,0]
    G = RGB[:,:,1]
    B = RGB[:,:,2]
    avg_r = np.average(R.flatten())
    avg_g = np.average(G.flatten())
    avg_b = np.average(B.flatten())

    HSV = color.rgb2hsv(RGB)
    S = HSV[:,:,1]
    avg_s = np.average(S.flatten())

    train_shape_feature.append([avg_r, avg_g, avg_b, avg_s, type_label])

generate_train("Clear", 0)
generate_train("Haze", 1)
generate_train("Underwater", 2)

In [6]:
# CREATE TRAIN CSV FEATURE MODEL 1
train_shape_feature = np.array(train_shape_feature)
np.savetxt(workdir + 'train_shape_feature_1.csv', train_shape_feature, delimiter=',', fmt='%s')

In [7]:
test_shape_feature = [["avg_r", "avg_g", "avg_b", "avg_s", "type_label"]]

# CUACA TEST FEATURE MODEL 1
def generate_test(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Test/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])
    R = RGB[:,:,0]
    G = RGB[:,:,1]
    B = RGB[:,:,2]
    avg_r = np.average(R.flatten())
    avg_g = np.average(G.flatten())
    avg_b = np.average(B.flatten())

    HSV = color.rgb2hsv(RGB)
    S = HSV[:,:,1]
    avg_s = np.average(S.flatten())

    test_shape_feature.append([avg_r, avg_g, avg_b, avg_s, type_label])

generate_test("Clear", 0)
generate_test("Haze", 1)
generate_test("Underwater", 2)

In [8]:
# CREATE TEST CSV FEATURE MODEL 1
test_shape_feature = np.array(test_shape_feature)
np.savetxt(workdir + 'test_shape_feature_1.csv', test_shape_feature, delimiter=',', fmt='%s')

## Feature Model 2 : Contrast, ASM, Homogeneity

In [9]:
train_shape_feature = [["contrast", "ASM", "homogeneity", "type_label"]]

# CUACA TRAIN FEATURE MODEL 2
def generate_train(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Train/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])
    
    BW = color.rgb2gray(RGB)
    BW = util.img_as_ubyte(BW)
    comat = feature.greycomatrix(BW, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                  levels=256, normed=True, symmetric=True)

    contrast = feature.greycoprops(comat, 'contrast')[0, 0]
    ASM = feature.greycoprops(comat, 'ASM')[0, 0]
    homogeneity = feature.greycoprops(comat, 'homogeneity')[0, 0]

    train_shape_feature.append([contrast, ASM, homogeneity, type_label])

generate_train("Clear", 0)
generate_train("Haze", 1)
generate_train("Underwater", 2)

In [10]:
# CREATE TRAIN CSV FEATURE MODEL 2
train_shape_feature = np.array(train_shape_feature)
np.savetxt(workdir + 'train_shape_feature_2.csv', train_shape_feature, delimiter=',', fmt='%s')

In [11]:
test_shape_feature = [["contrast", "ASM", "homogeneity", "type_label"]]

# CUACA TEST FEATURE MODEL 2
def generate_test(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Test/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])

    BW = color.rgb2gray(RGB)
    BW = util.img_as_ubyte(BW)
    comat = feature.greycomatrix(BW, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                  levels=256, normed=True, symmetric=True)

    contrast = feature.greycoprops(comat, 'contrast')[0, 0]
    ASM = feature.greycoprops(comat, 'ASM')[0, 0]
    homogeneity = feature.greycoprops(comat, 'homogeneity')[0, 0]

    test_shape_feature.append([contrast, ASM, homogeneity, type_label])

generate_test("Clear", 0)
generate_test("Haze", 1)
generate_test("Underwater", 2)

In [12]:
# CREATE TEST CSV FEATURE MODEL 2
test_shape_feature = np.array(test_shape_feature)
np.savetxt(workdir + 'test_shape_feature_2.csv', test_shape_feature, delimiter=',', fmt='%s')

## PCA Preprocessing

In [13]:
def generate_sq(type_str, subfolder='Train'):
  files = glob.glob(workdir + f'Cuaca/{subfolder}/{type_str}/**/*')
  
  augs = [io.imread(f) for f in files]
  sq = []
  for i in augs:
      h, w = i.shape[0], i.shape[1]
      px = min(h, w)
      sq.append(i[(h-px)//2:((h-px)//2)+px, (w-px)//2:((w-px)//2)+px])
  return sq

In [14]:
square = {
    'Clear': generate_sq('Clear'),
    'Haze': generate_sq('Haze'),
    'Underwater': generate_sq('Underwater'),
}

In [15]:
minpx = min((d.shape[0] for s in square for d in square[s]))

def resizeall(dataset, px):
  return [resize(s, (px, px), anti_aliasing=True) for s in dataset]

dataset = {k: util.img_as_ubyte(resizeall(square[k], 50)) for k in square}

In [16]:
dataset.keys()

dict_keys(['Clear', 'Haze', 'Underwater'])

Dataset final ada di

`dataset['Clear']`, `dataset['Haze']`, dan `dataset['Underwater']`

# Kode PCA

In [17]:
# Input : Matrix
# Output : Matrix transformasi linier ke basis yang baru, matriks proyeksi
def PCA_proj(X):
  cov_mat = np.cov(X.T)

  eig_vals, eig_vecs = np.linalg.eig(cov_mat)
  eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]
  eig_pairs.sort(key = lambda x : x[0], reverse = True)

  exp_var_perc = 0.97
  tot = sum(eig_vals)
  var_exp = [(i / tot) for i in sorted(eig_vals,  reverse = True)]
  cum_var_exp = np.cumsum(var_exp)

  num_vec_to_keep = 0
  for i, percentage in enumerate(cum_var_exp):
    if percentage > exp_var_perc:
      num_vec_to_keep = i + 1
      break
  
  num_features = X.shape[1]
  proj_mat = eig_pairs[0][1].reshape(num_features, 1)
  for eig_vec_idx in range(1, num_vec_to_keep):
    proj_mat = np.hstack((proj_mat, eig_pairs[eig_vec_idx][1].reshape(num_features, 1)))

  return proj_mat

## Feature SVM Processing Code

In [18]:
from sklearn import svm, metrics
import pandas as pd
import numpy as np

def svm_process(train_file, test_file):
  train_df = pd.read_csv(workdir + train_file)
  train_df = train_df.sample(frac=1)

  train_label = train_df["type_label"]
  train_label = pd.DataFrame(train_label).to_numpy()

  train_df.drop(labels="type_label", axis=1, inplace=True)
  train_shape_feature = pd.DataFrame(train_df).to_numpy()

  test_df = pd.read_csv(workdir + test_file)
  test_df = test_df.sample(frac=1)

  test_label = test_df["type_label"]
  test_label = pd.DataFrame(test_label).to_numpy()

  test_df.drop(labels="type_label", axis=1, inplace=True)
  test_shape_feature = pd.DataFrame(test_df).to_numpy()

  train_label = train_label.ravel()
  test_label = test_label.ravel()

  clf = svm.SVC(gamma='scale')
  clf.fit(train_shape_feature, train_label)
  svm_label = clf.predict(test_shape_feature)
  acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
  cm_svm = metrics.confusion_matrix(svm_label, test_label)
  print("Accuracy", acc)
  print("Confusion Matrix")
  print(cm_svm)

In [19]:
print("Feature 1")
svm_process("train_shape_feature_1.csv", "test_shape_feature_1.csv")
print()
print("Feature 2")
svm_process("train_shape_feature_2.csv", "test_shape_feature_2.csv")

Feature 1
Accuracy 0.7333333333333333
Confusion Matrix
[[3 2 0]
 [2 3 0]
 [0 0 5]]

Feature 2
Accuracy 0.6666666666666666
Confusion Matrix
[[5 0 2]
 [0 5 3]
 [0 0 0]]


# PCA SVM Processing Code

In [20]:
# Memang lama
from sklearn.preprocessing import StandardScaler

# Train Preprocessing
merged = np.concatenate((dataset['Clear'], dataset['Haze'], dataset['Underwater']), axis = 0)
merged.shape

train_label = [0 for i in range(60)] + [1 for i in range(60)] + [2 for i in range(60)]

merged = np.array([k.flatten() for k in merged])
merged_std = StandardScaler().fit_transform(merged)

proj_mat = PCA_proj(merged_std)
pca_train = merged_std.dot(proj_mat)
pca_train = pca_train.real

In [21]:
# Test Preprocessing
# testsq = generate_sq('*', 'Test') # di dalam collab, ternyata milihnya random
testsq = generate_sq('Clear', 'Test')
testsq.extend(generate_sq('Haze', 'Test'))
testsq.extend(generate_sq('Underwater', 'Test'))

testset = util.img_as_ubyte(resizeall(testsq, 50))
testset = np.array([k.flatten() for k in testset])

test_label = [0 for i in range(5)] + [1 for i in range(5)] + [2 for i in range(5)]
testset_std = StandardScaler().fit_transform(testset)

pca_test = testset_std.dot(proj_mat)
pca_test = pca_test.real

In [22]:
# SVM
train_label = np.array(train_label)
test_label = np.array(test_label)

train_label = train_label.ravel()
test_label = test_label.ravel()

clf = svm.SVC(gamma='scale')
clf.fit(pca_train, train_label)
svm_label = clf.predict(pca_test)

acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print("Accuracy", acc)
print("Confusion Matrix")
print(cm_svm)

Accuracy 0.8666666666666667
Confusion Matrix
[[3 0 0]
 [2 5 0]
 [0 0 5]]


# Improvement
Gabung 2 fitur + PCA fiturnya

# Gabung 2 Fitur

In [23]:
train_shape_feature = [["avg_r", "avg_g", "avg_b", "avg_s", "contrast", "ASM", "homogeneity", "type_label"]]

# CUACA TRAIN FEATURE MODEL GABUNGAN
def generate_train(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Train/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])
    R = RGB[:,:,0]
    G = RGB[:,:,1]
    B = RGB[:,:,2]
    avg_r = np.average(R.flatten())
    avg_g = np.average(G.flatten())
    avg_b = np.average(B.flatten())

    HSV = color.rgb2hsv(RGB)
    S = HSV[:,:,1]
    avg_s = np.average(S.flatten())

    BW = color.rgb2gray(RGB)
    BW = util.img_as_ubyte(BW)
    comat = feature.greycomatrix(BW, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                  levels=256, normed=True, symmetric=True)

    contrast = feature.greycoprops(comat, 'contrast')[0, 0]
    ASM = feature.greycoprops(comat, 'ASM')[0, 0]
    homogeneity = feature.greycoprops(comat, 'homogeneity')[0, 0]

    train_shape_feature.append([avg_r, avg_g, avg_b, avg_s, contrast, ASM, homogeneity, type_label])

generate_train("Clear", 0)
generate_train("Haze", 1)
generate_train("Underwater", 2)

In [24]:
# CREATE TRAIN CSV FEATURE MODEL GABUNGAN
train_shape_feature = np.array(train_shape_feature)
np.savetxt(workdir + 'train_shape_feature_3.csv', train_shape_feature, delimiter=',', fmt='%s')

In [25]:
test_shape_feature = [["avg_r", "avg_g", "avg_b", "avg_s", "contrast", "ASM", "homogeneity", "type_label"]]

# CUACA TEST FEATURE MODEL GABUNGAN
def generate_test(type_str, type_label):
  files_tr_clear = glob.glob(workdir + "Cuaca/Test/{}/**/*".format(type_str))
  N = len(files_tr_clear)

  for k in range(N):
    RGB = io.imread(files_tr_clear[k])
    R = RGB[:,:,0]
    G = RGB[:,:,1]
    B = RGB[:,:,2]
    avg_r = np.average(R.flatten())
    avg_g = np.average(G.flatten())
    avg_b = np.average(B.flatten())

    HSV = color.rgb2hsv(RGB)
    S = HSV[:,:,1]
    avg_s = np.average(S.flatten())

    BW = color.rgb2gray(RGB)
    BW = util.img_as_ubyte(BW)
    comat = feature.greycomatrix(BW, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                  levels=256, normed=True, symmetric=True)

    contrast = feature.greycoprops(comat, 'contrast')[0, 0]
    ASM = feature.greycoprops(comat, 'ASM')[0, 0]
    homogeneity = feature.greycoprops(comat, 'homogeneity')[0, 0]

    test_shape_feature.append([avg_r, avg_g, avg_b, avg_s, contrast, ASM, homogeneity, type_label])

generate_test("Clear", 0)
generate_test("Haze", 1)
generate_test("Underwater", 2)

In [26]:
# CREATE TEST CSV FEATURE MODEL GABUNGAN
test_shape_feature = np.array(test_shape_feature)
np.savetxt(workdir + 'test_shape_feature_3.csv', test_shape_feature, delimiter=',', fmt='%s')

In [27]:
train_df = pd.read_csv(workdir + "train_shape_feature_3.csv")
train_df = train_df.sample(frac=1)

train_label = train_df["type_label"]
train_label = pd.DataFrame(train_label).to_numpy()

train_df.drop(labels="type_label", axis=1, inplace=True)
train_shape_feature = pd.DataFrame(train_df).to_numpy()

test_df = pd.read_csv(workdir + "test_shape_feature_3.csv")
test_df = test_df.sample(frac=1)

test_label = test_df["type_label"]
test_label = pd.DataFrame(test_label).to_numpy()

test_df.drop(labels="type_label", axis=1, inplace=True)
test_shape_feature = pd.DataFrame(test_df).to_numpy()

In [28]:
from sklearn import svm, metrics

train_label = train_label.ravel()
test_label = test_label.ravel()

clf = svm.SVC(gamma='scale')
clf.fit(train_shape_feature, train_label)
svm_label = clf.predict(test_shape_feature)

In [29]:
acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(acc)
print(cm_svm)

1.0
[[5 0 0]
 [0 5 0]
 [0 0 5]]


# PCA

In [30]:
train_std = StandardScaler().fit_transform(train_shape_feature)
proj_mat = PCA_proj(train_std)
train_std = train_std.dot(proj_mat)
print(train_std.shape)

test_std = StandardScaler().fit_transform(test_shape_feature)
test_std = test_std.dot(proj_mat)
print(test_std.shape)

(180, 5)
(15, 5)


In [31]:
from sklearn import svm, metrics

train_label = train_label.ravel()
test_label = test_label.ravel()

clf = svm.SVC(gamma='scale')
clf.fit(train_shape_feature, train_label)
svm_label = clf.predict(test_shape_feature)

In [32]:
acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
cm_svm = metrics.confusion_matrix(svm_label, test_label)
print(acc)
print(cm_svm)

1.0
[[5 0 0]
 [0 5 0]
 [0 0 5]]


# Cross Validation Hanya Gabung Fitur

In [33]:
total = 0

NUM_LOOP = 10000
for i in range(NUM_LOOP):
  train_df = pd.read_csv(workdir + "train_shape_feature_3.csv")
  train_df = train_df.sample(frac=1) # randomized
  
  N = len(train_df)
  n_train = int(N * .8)

  train_label = train_df["type_label"][:n_train]
  train_label = pd.DataFrame(train_label).to_numpy()

  test_label = train_df["type_label"][n_train:]
  test_label = pd.DataFrame(test_label).to_numpy()

  train_df.drop(labels="type_label", axis=1, inplace=True)
  train_shape_feature = pd.DataFrame(train_df[:n_train]).to_numpy()
  test_shape_feature = pd.DataFrame(train_df[n_train:]).to_numpy()

  train_label = train_label.ravel()
  test_label = test_label.ravel()

  clf = svm.SVC(gamma='scale')
  clf.fit(train_shape_feature, train_label)
  svm_label = clf.predict(test_shape_feature)

  acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
  cm_svm = metrics.confusion_matrix(svm_label, test_label)
  total += acc

print("Average Accuracy:")
print(total / NUM_LOOP)

Average Accuracy:
0.8175277777777659


# Cross Validation Gabung Fitur + PCA

In [34]:
total = 0

NUM_LOOP = 10000
for i in range(NUM_LOOP):
  train_df = pd.read_csv(workdir + "train_shape_feature_3.csv")
  train_df = train_df.sample(frac=1)

  N = len(train_df)
  n_train = int(N * .8)
  n_test = int(N * .2)

  train_label = train_df["type_label"][:n_train]
  train_label = pd.DataFrame(train_label).to_numpy()

  test_label = train_df["type_label"][n_train:]
  test_label = pd.DataFrame(test_label).to_numpy()

  train_df.drop(labels="type_label", axis=1, inplace=True)
  train_shape_feature = pd.DataFrame(train_df[:n_train]).to_numpy()
  test_shape_feature = pd.DataFrame(train_df[n_train:]).to_numpy()


  train_std = StandardScaler().fit_transform(train_shape_feature)
  proj_mat = PCA_proj(train_std)
  train_std = train_std.dot(proj_mat)

  test_std = StandardScaler().fit_transform(test_shape_feature)
  test_std = test_std.dot(proj_mat)
  
  train_shape_feature = train_std
  test_shape_feature = test_std


  from sklearn import svm, metrics

  train_label = train_label.ravel()
  test_label = test_label.ravel()

  clf = svm.SVC(gamma='scale')
  clf.fit(train_shape_feature, train_label)
  svm_label = clf.predict(test_shape_feature)

  acc = sum(1 if svm_label[i] == test_label[i] else 0 for i in range(len(test_label))) / len(test_label)
  cm_svm = metrics.confusion_matrix(svm_label, test_label)

  total += acc

print("Average Accuracy:")
print(total / NUM_LOOP)

Average Accuracy:
0.891297222222231


# Neural Network with Keras

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [36]:
classifier = Sequential([
    Conv2D(32, (3, 3), input_shape=(50, 50, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=3, activation='softmax')
])

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/TL4 PengCit/Cuaca/Train', target_size=(50, 50), batch_size=1, class_mode='categorical')
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/TL4 PengCit/Cuaca/Test', target_size=(50, 50), batch_size=1, class_mode='categorical')

Found 180 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [38]:
classifier.fit(training_set, steps_per_epoch=18, epochs=10, validation_data=test_set, validation_steps=3, verbose=1)

Epoch 1/10
18/18 [==============================] - 0s 26ms/step - loss: 887.7383 - accuracy: 0.4444 - val_loss: 189.5140 - val_accuracy: 0.6667
Epoch 2/10
18/18 [==============================] - 0s 17ms/step - loss: 39.4285 - accuracy: 0.8333 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
18/18 [==============================] - 0s 16ms/step - loss: 144.6222 - accuracy: 0.6111 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
18/18 [==============================] - 0s 16ms/step - loss: 13.8017 - accuracy: 0.8889 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
18/18 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 6.7328 - val_accuracy: 0.6667
Epoch 6/10
18/18 [==============================] - 0s 19ms/step - loss: 4.6652 - accuracy: 0.8889 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
18/18 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 